In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler

import pickle

In [2]:
df  = pd.read_csv('Churn_Modelling.csv')

In [3]:
dfcopy =df.copy()

In [4]:
dfcopy.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## preprocessing the data 
## dropping the irrelevant feature
dfcopy = dfcopy.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [6]:
dfcopy.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [7]:
## Encode categorical variable
label_encoder_gender = LabelEncoder()
dfcopy['Gender'] = label_encoder_gender.fit_transform(dfcopy['Gender'])



In [8]:
## use One hot encoding for Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(dfcopy[['Geography']])

In [9]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [11]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [12]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [13]:
## combining one hot encoder columns with the original data
dfcopy = pd.concat([dfcopy.drop('Geography',axis=1),geo_encoded_df],axis=1)


In [14]:
dfcopy.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
### save the encoder and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [18]:
## divide the data into dependent feature and independent feature
X = dfcopy.drop('Exited',axis=1)
y = dfcopy['Exited']

## split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=42)

## scaling the feature 
scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [19]:
## saving scaler inthe pickle file

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN Implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [30]:
(X_train.shape[1],)

(12,)

In [34]:
## build our model
Model =Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),  ## HL1
    Dense(32,activation='relu') ,## HL2
    Dense(1,activation='sigmoid')
]
)

In [36]:
Model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

In [39]:
### compile the model

Model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
### setup the tensorboard
log_dir = "logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback =TensorBoard(log_dir=log_dir,histogram_freq=1)

In [42]:
## setup earlystopping
early_stopping_callback =EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [43]:
## train the model 
history = Model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 4ms/step - loss: 0.3984 - accuracy: 0.8313 - val_loss: 0.3540 - val_accuracy: 0.8505
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3565 - accuracy: 0.8545 - val_loss: 0.3394 - val_accuracy: 0.8590
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3511 - accuracy: 0.8589 - val_loss: 0.3389 - val_accuracy: 0.8600
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3408 - accuracy: 0.8600 - val_loss: 0.3410 - val_accuracy: 0.8610
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3395 - accuracy: 0.8597 - val_loss: 0.3651 - val_accuracy: 0.8625
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3382 - accuracy: 0.8616 - val_loss: 0.3553 - val_accuracy: 0.8560
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3361 - accuracy: 0.8600 - val_loss: 0.3404 - val_accuracy: 0.86

In [44]:
Model.save('model.h5')

c:\ANN_Classification\ann_classification\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
## load tensorboard Extension
%load_ext tensorboard

In [51]:
%tensorboard --logdir logs/fit20250707-202516

Launching TensorBoard...